In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import seaborn as sns
import matplotlib.pyplot as plt

import gc

from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
import lightgbm as lgb

from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import MinMaxScaler
# Any results you write to the current directory are saved as output.

In [ ]:
#reading test data and calculating the shortest distance over the earth’s surface
test = pd.read_csv("../input/test.csv")
gc.collect()
R = 6371e3
a1 = np.radians(test['pickup_latitude'])
a2 = np.radians(test['dropoff_latitude'])
da = np.radians(test['dropoff_latitude']-test['pickup_latitude'])
dl = np.radians(test['dropoff_longitude']-test['pickup_longitude'])

test = test.drop(columns = ['pickup_latitude','dropoff_latitude','pickup_longitude','dropoff_longitude'])

a = np.sin(da/2) * np.sin(da/2) + np.cos(a1) * np.cos(a2) * np.sin(dl/2) * np.sin(dl/2)
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
d = R * c;

del R,c,a,a1,a2,da,dl

test['Distance']= pd.Series(d)
del d
gc.collect()

In [ ]:
# Reading File
train_path  = '../input/train.csv'

# Set columns to most suitable type to optimize for memory usage
traintypes = {'fare_amount': 'float32',
              'pickup_datetime': 'str', 
              'pickup_longitude': 'float32',
              'pickup_latitude': 'float32',
              'dropoff_longitude': 'float32',
              'dropoff_latitude': 'float32',
              'passenger_count': 'uint8'}

cols = list(traintypes.keys())

dftemp = pd.read_csv(train_path, usecols=cols, dtype=traintypes, nrows = 500000)
#50000000
gc.collect()

In [ ]:
# Save into feather format, about 1.5Gb. 
dftemp.to_feather('nyc_taxi_data_raw.feather')
del dftemp,traintypes,cols
gc.collect()

In [ ]:
# load the same dataframe next time directly, without reading the csv file again!
df = pd.read_feather('nyc_taxi_data_raw.feather')

# It took less than one tenth of time to read the file

In [ ]:
#the shortest distance over the earth’s surface
gc.collect()
R = 6371e3
a1 = np.radians(df['pickup_latitude'])
a2 = np.radians(df['dropoff_latitude'])
da = np.radians(df['dropoff_latitude']-df['pickup_latitude'])
dl = np.radians(df['dropoff_longitude']-df['pickup_longitude'])

df = df.drop(columns = ['pickup_latitude','dropoff_latitude','pickup_longitude','dropoff_longitude'])

a = np.sin(da/2) * np.sin(da/2) + np.cos(a1) * np.cos(a2) * np.sin(dl/2) * np.sin(dl/2)
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
d = R * c;

del R,c,a,a1,a2,da,dl

df['Distance']= pd.Series(d)
del d
gc.collect()

In [ ]:
#fig, (ax1,ax2) = plt.subplots(figsize=(12,9),ncols=2, nrows=1)
#sns.regplot(x='passenger_count'^2,y='fare_amount',data=df)
#gc.collect()
#sns.regplot(x='Distance',y='fare_amount',data=df)
#gc.collect()
df = df.dropna()
sns.distplot(df['passenger_count'])
gc.collect()

In [ ]:
df = df[~(df[['passenger_count']] == 0).any(axis=1)]
scaler = MinMaxScaler()
gc.collect()

In [ ]:
sns.set(style="whitegrid")
sns.boxplot(x="passenger_count", y="Distance", data=df)
plt.grid(True)
plt.show()
gc.collect()

In [ ]:
#Linear regression, Lasso, LGB, XGBoost tried 

y = df['fare_amount']
#df['Distance_per_passenger'] = df['Distance']/df['passenger_count']
x = df.drop(columns=['pickup_datetime','fare_amount'])
#x = x.values
scaler.fit_transform(x) 
gc.collect()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.20)
del x,y
gc.collect()
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV
rfc = RandomForestRegressor() 
# Use a grid over parameters of interest
param_grid = { 
           "n_estimators" : [ 36, 45, 54, 63,100],
           "max_depth" : [ 5, 10, 15, 20,40],
            "min_samples_leaf" : [1,5,10,20,40]
           }
model = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 10)
model.fit(X_train, y_train)
print(model.best_params_)
y_pred = model.predict(X_val)
print('The rmse of prediction is:', mean_squared_error(y_val, y_pred) ** 0.5)
gc.collect()



In [ ]:
x_test = test.drop(columns=['key','pickup_datetime'])
scaler.transform(x_test)
gc.collect()

In [ ]:
y_test = model.predict(x_test)
y_test =  y_test
df = pd.DataFrame()
df["key"] = test["key"]
df["fare_amount"] = y_test
df.to_csv("sample_submission.csv", index = False)